In [1]:
%tensorflow_version 1.x
import tensorflow as tf
from keras.datasets import cifar10
import numpy as np
import keras
from keras.layers import AveragePooling2D, Flatten, Dense
from datetime import datetime
from numpy.random import seed
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

seed(1)
tf.random.set_random_seed(seed=2)


batch_size = 32
epochs = 200
data_augmentation = False
num_classes = 10

subtract_pixel_mean = True

n = 3
depth = n*6+2 # DEPTH = 20


# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input shape
input_shape = x_train.shape[1:]

# Nomalize data
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# Substract pixel mean
if subtract_pixel_mean:
  x_train_mean = np.mean(x_train, axis=0)
  x_train -= x_train_mean
  x_test -= x_train_mean

print('x_train_shape: ', x_train.shape)
print('train samples: ', x_train.shape[0])
print('test samples: ' , x_test.shape[0])


# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape: ', y_train.shape)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 120:
        lr *= 0.5e-3
    elif epoch > 100:
        lr *= 1e-3
    elif epoch > 80:
        lr *= 1e-2
    elif epoch > 60:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

def conv2d(x, W, strides, padding):
  return tf.nn.conv2d(x, W, strides= strides, padding=padding)

def weights(kernel_size, in_channels, out_channels, name=None):
  return tf.Variable(tf.keras.initializers.he_normal()([kernel_size, kernel_size, in_channels, out_channels]), name=name)

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
  

  x=inputs
  in_channels = x.get_shape().as_list()[3]
  beta = tf.Variable(tf.zeros([num_filters]), name="beta")
  if conv_first:
    x = conv2d(x,
                weights(kernel_size,in_channels,out_channels = num_filters),
                strides,
                padding = 'SAME')
    if batch_normalization:
      mean, variance = tf.nn.moments(x, axes=[0,1,2])
      x = tf.nn.batch_normalization(x, mean, variance, offset=None, scale=None, variance_epsilon = 0.000)
    if activation is not None:
      x = tf.nn.relu(x + beta)
  else:
    # batch normalization first
    if batch_normalization:
      mean, variance = tf.nn.moments(x, axes=[0,1,2])
      x = tf.nn.batch_normalization(x, mean, variance, offset=None, scale=None, variance_epsilon = 0.0001)
    if activation is not None:
      x = tf.nn.relu(x + beta)
    # Then convolution
    x = conv2d(x)+ beta

  return x



def resnet_v1(input_tensor, depth, num_classes=10):
  """ResNet Version 1 Model builder [a]

  Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
  Last ReLU is after the shortcut connection.
  At the beginning of each stage, the feature map size is halved (downsampled)
  by a convolutional layer with strides=2, while the number of filters is
  doubled. Within each stage, the layers have the same number filters and the
  same number of filters.
  Features maps sizes:
  stage 0: 32x32, 16
  stage 1: 16x16, 32
  stage 2:  8x8,  64
  The Number of parameters of Rest20 is approx 0.27M


  # Arguments
      input_tensor (tensor): shape of input image tensor
      depth (int): number of core convolutional layers
      num_classes (int): number of classes (CIFAR10 has 10)

  # Returns
      model (Model): tensorflow graph
  """
  if (depth - 2) % 6 != 0:
      raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
  # Start model definition.
  num_filters = 16
  num_res_blocks = int((depth - 2) / 6)

  # inputs: placeholder 

  x = resnet_layer(input_tensor)
  # Instantiate the stack of residual units
  for stack in range(3):
      for res_block in range(num_res_blocks):
          strides = 1
          if stack > 0 and res_block == 0:  # first layer but not first stack
              strides = 2  # downsample
          y = resnet_layer(inputs=x,
                            num_filters=num_filters,
                            strides=strides)
          y = resnet_layer(inputs=y,
                            num_filters=num_filters,
                            activation=None)
          if stack > 0 and res_block == 0:  # first layer but not first stack
              # linear projection residual shortcut connection to match
              # changed dims
              x = resnet_layer(inputs=x,
                                num_filters=num_filters,
                                kernel_size=1,
                                strides=strides,
                                activation=None,
                                batch_normalization=False)
          x = keras.layers.add([x, y])
          x = tf.nn.relu(x)
      num_filters *= 2

  # Add classifier on top.
  # v1 does not use BN after last shortcut connection-ReLU
  x = AveragePooling2D(pool_size=8)(x)
  y = Flatten()(x)
  outputs = Dense(num_classes,
                  kernel_initializer='he_normal')(y)

  return outputs, y

      

      
  


Using TensorFlow backend.


x_train_shape:  (50000, 32, 32, 3)
train samples:  50000
test samples:  10000
y_train shape:  (50000, 10)


In [0]:
def save_graph(name):
  root_logdir = "log"
  logdir = "{}/run-{}/".format(root_logdir, name)
  file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

name = 'grafo_RESNET20'
save_graph(name)


In [0]:
def tensorboard():
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip

  LOG_DIR = './log'
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR)
  )

  get_ipython().system_raw('./ngrok http 6006 &')

  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# data augmentation
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

def data_augmetation(datagen,x_b,y_b):
  for x_batch, y_batch in datagen.flow(x_b, y_b, batch_size=x_b.shape[0]):
    break
  return x_batch, y_batch

  


In [0]:
height, width, channels = input_shape

def crear_batch(x_train, y_train, i, batch_size):
    x_batch = x_train[i*batch_size: (i+1)*batch_size]
    y_batch = y_train[i*batch_size:(i+1)*batch_size]
    return x_batch, y_batch

def evaluar_test(x_test,y_test,x,y,lr, lr_value,accuracy,cost, k=10):
  n = len(x_test)
  batch = int(n/k)
  accuracy_test = 0
  loss_test = 0
  for i in range(k):
    x_batch = x_test[i*batch:(i+1)*batch]
    y_batch = y_test[i*batch:(i+1)*batch]
    feed_dict_batch_eval = {x: x_batch, y: y_batch, lr: lr_value} 
    
    accuracy_test += accuracy.eval(feed_dict_batch_eval)/k
    loss_test += cost.eval(feed_dict_batch_eval)/k
  return accuracy_test, loss_test

def get_variable_value(tensor_name, sess):
  t = tf.get_default_graph().get_tensor_by_name(tensor_name)
  r = sess.run(t)
  return r


def train_cnn(epochs,x_train,y_train, checkpoint=None):    
    tf.reset_default_graph()    
    # Entrada grafo
    start = datetime.now()  
    

    x = tf.placeholder(tf.float32, shape=(None, height, width, channels))
    y = tf.placeholder(tf.float32, [None, num_classes])
    lr = tf.placeholder(tf.float32, [])
    output, _ = resnet_v1(x, depth, num_classes=10)
    y_ = tf.nn.softmax(output, name='prediction')
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y), name='loss')
    optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(cost)
    correct = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))  
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
    
    # nodo para inicializar variables
    init_op = tf.global_variables_initializer()


    #  Cargar pesos
    if checkpoint == None:
      saver = tf.train.Saver()
    else:
      variables = [v for v in tf.global_variables()]

      vars_to_restore_dict = {}
      for v in variables:
        vars_to_restore_dict[v.name[:-2]] = v
      saver = tf.train.Saver(vars_to_restore_dict)

    #file_writer = tf.summary.FileditareWriter(logdir, tf.get_default_graph())
    vector_costos_train = np.zeros([epochs])
    vector_costos_test = np.zeros([epochs])
    vector_acc_test = np.zeros([epochs])
    vector_acc_train = np.zeros([epochs])
    
    

    with tf.Session() as sess:
        # restaurar pesos
        if checkpoint == None:
          sess.run(init_op) #cuando se realiza inicializacion aleatoria
        else:
          saver.restore(sess, checkpoint) # restaurar desde checkpoint

        # parametros iniciales 
        best_epoch_loss = 10000        
        best_acc_test = 0
        

        
        
        for epoch in range(epochs):
            epoch_loss = 0
            acc_train = 0
            lr_value = lr_schedule(epoch)
            # if random state= None, Shuffle randomly each time.
            x_train, y_train = shuffle(x_train, y_train, random_state=None)        
              

            
            for i in range(int(len(x_train)/batch_size)):
                x_batch, y_batch = crear_batch(x_train,y_train,i,batch_size)
                x_batch, y_batch = data_augmetation(datagen,x_batch, y_batch)
                feed_dict_batch = {x: x_batch, y: y_batch, lr: lr_value}

                _, c = sess.run([optimizer, cost], feed_dict_batch)
                feed_dict_batch_eval = {x: x_batch, y: y_batch, lr: lr_value}              
                acc_batch = accuracy.eval(feed_dict_batch_eval)
                epoch_loss += c
                acc_train += acc_batch
                
            accuracy_test, loss_test =  evaluar_test(x_test,y_test,x,y,lr, lr_value,accuracy,cost,k=10)
            

            print('Epoch %i de %i Loss train: %.4f Loss test: %.4f Accuracy train: %.4f Accuracy test: %.4f '%(epoch,epochs,epoch_loss/(int(len(x_train)/batch_size)),
                                                                                                                   loss_test,
                                                                                                                   acc_train/(int(len(x_train)/batch_size)),
                                                                                                                   accuracy_test))         
            vector_costos_train[epoch] = epoch_loss/(int(len(x_train)/batch_size))
            vector_costos_test[epoch] = loss_test
            vector_acc_train[epoch] = acc_train/(int(len(x_train)/batch_size))        
            vector_acc_test[epoch] = accuracy_test 

            if (accuracy_test > best_acc_test):
              best_acc_test = accuracy_test
              save_path = saver.save(sess, "checkpoint{}.ckpt".format(epoch))


              
                
            if (epoch%10 == 0):
              difference = datetime.now() - start
              print('Tiempo: ',difference)


        save_path = saver.save(sess,"my_model_final.ckpt")
        difference = datetime.now() - start
        print('Tiempo: ',difference)
        print('El mejor accuracy en test es: ',best_acc_test)
      
        return vector_costos_train, vector_costos_test, vector_acc_test, vector_acc_train




In [0]:
#epochs = 200
#train_cnn(epochs,x_train,y_train, checkpoint=None)

In [7]:
x = tf.placeholder(tf.float32, shape=(None, height, width, channels))
y = tf.placeholder(tf.float32, [None, num_classes])
lr = tf.placeholder(tf.float32, [])
output, fv = resnet_v1(x, depth, num_classes=10)
y_ = tf.nn.softmax(output, name='prediction')
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y), name='loss')
optimizer = tf.train.AdamOptimizer(learning_rate = lr).minimize(cost)
correct = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))  
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
  



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
      tf.train.Saver().restore(sess, '/content/drive/My Drive/Memoria_Ingenieria_electrica/Codigo/CODIGO MEMORIA/CNN/RESNET/weights_data_augmentation/checkpoint96.ckpt') # restaurar desde checkpoint
      feed_dict_batch_eval = {x: x_train, y: y_train, lr: 0.0}              
      fv_eval_train = fv.eval(feed_dict_batch_eval)
      feed_dict_batch_eval = {x: x_test, y: y_test, lr: 0.0}              
      fv_eval_test = fv.eval(feed_dict_batch_eval)
      print(accuracy.eval(feed_dict_batch_eval))
    
      


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Memoria_Ingenieria_electrica/Codigo/CODIGO MEMORIA/CNN/RESNET/weights_data_augmentation/checkpoint96.ckpt
0.9075


In [0]:
np.save('/content/drive/My Drive/Memoria_Ingenieria_electrica/Codigo/CODIGO MEMORIA/CNN/RESNET/fv_RESNET20_da/fv_RESNET20_da_train.npy', fv_eval_train)
np.save('/content/drive/My Drive/Memoria_Ingenieria_electrica/Codigo/CODIGO MEMORIA/CNN/RESNET/fv_RESNET20_da/fv_RESNET20_da_test.npy', fv_eval_test)

## SVM with feature vector

In [10]:
from sklearn.model_selection import train_test_split
import datetime
from sklearn import svm

(_, y_train), (_, y_test) = cifar10.load_data()

def extractor_clasificador():

  print('data train: ',fv_eval_train.shape)
  print('data test: ', fv_eval_test.shape)

  X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(fv_eval_train, y_train, test_size=0.33, random_state=42)


  C = [0.01,0.1,1, 10,100]
  

  first_time = datetime.datetime.now()

  for c in C:
      svc = svm.SVC(kernel = 'rbf', C = c).fit(X_train_a, y_train_a.ravel())  
      # model accuracy for X_test   
      accuracy = svc.score(X_test_a, y_test_a.ravel())
      accuracy_val = svc.score(fv_eval_test, y_test.ravel())
      print('Con c: ',c, 'el accuracy validacion es : ', accuracy, 'accuracy test es: ', accuracy_val)

  later_time = datetime.datetime.now()
  difference = later_time - first_time
  print('Tiempo de ejecucion: ', difference)

extractor_clasificador()

data train:  (50000, 64)
data test:  (10000, 64)
Con c:  0.01 el accuracy validacion es :  0.9733333333333334 accuracy test es:  0.906
Con c:  0.1 el accuracy validacion es :  0.9767272727272728 accuracy test es:  0.9092
Con c:  1 el accuracy validacion es :  0.9766060606060606 accuracy test es:  0.9075
Con c:  10 el accuracy validacion es :  0.9723636363636363 accuracy test es:  0.9025
Con c:  100 el accuracy validacion es :  0.967939393939394 accuracy test es:  0.9002
Tiempo de ejecucion:  0:04:04.439703
